In [ ]:
# NVIDIA gpu check
!nvidia-smi

Sat Jan 20 20:03:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Setup (install and imports)

In [1]:
from IPython.display import clear_output

# LIBRARIES TO INSTALL
!pip install ultralytics
!pip install clearml
clear_output()

In [2]:
# IMPORTS
import os
import shutil
from ultralytics import YOLO
import random
import locale
from google.colab import files

#clearml imports
import clearml
from clearml import Task
import tensorboard

### Setup (clearml)

In [3]:
clearml.browser_login()

task = Task.init(
    project_name = "test",
    task_name = "Leefilter 10k"
)

model_variant = "yolov8n"
task.set_parameter("modal_variant", model_variant)

# number of images to pick to perform the training (the will be splitted
# into 70% training and 30% for validation)
nmr_images = 10000
task.set_parameter("nmr_images", nmr_images)

# Load the YOLOv8 model
model = YOLO(f"{model_variant}.pt") # load a pretrained model

args = dict(data    = "/content/SAR-Ship-Dataset/data_leefilter.yaml",
            epochs  = 100,
            imgsz   = 256,
            device  = 0,
            batch   = 256)

<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀
ClearML Task: created new task id=8a7990ba9abb4c50ab2348702dfc825f
2024-01-24 10:36:30,626 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/20fd35ca22f44c058a4937b57d40b24e/experiments/8a7990ba9abb4c50ab2348702dfc825f/output/log


100%|██████████| 6.23M/6.23M [00:00<00:00, 81.1MB/s]


2024-01-24 10:36:42,774 - clearml.model - INFO - Selected model id: 8f334a97307d474fa91242f6c43022bb


### Setup (folder training and validation)

In [4]:
clone_repo = "https://github.com/eboss-dev/SAR-Ship-Dataset"
list_dataset = ["ship_dataset_preprocessed", "ship_dataset_v0",
                "ship_dataset_leefilter"]
!git clone {clone_repo}

Cloning into 'SAR-Ship-Dataset'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 45 (delta 17), reused 22 (delta 4), pack-reused 0
Receiving objects: 100% (45/45), 9.96 KiB | 9.96 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Filtering content: 100% (3/3), 692.39 MiB | 58.85 MiB/s, done.


In [5]:
# remove unnecessary file
!rm ./SAR-Ship-Dataset/README.md


for to_unzip in list_dataset:
  !unzip ./SAR-Ship-Dataset/{to_unzip}.zip -d ./SAR-Ship-Dataset/
  # Once extracted remove the zip file downloaded from the github repo
  !rm ./SAR-Ship-Dataset/{to_unzip}.zip

clear_output()

In [ ]:
#!rm -rf "/content/SAR-Ship-Dataset/"

In [6]:
DATASET_PATH = "/content/SAR-Ship-Dataset/"

for dataset in list_dataset:

  _, _, files = next(os.walk(os.path.join(DATASET_PATH, dataset)))
  file_count = len(files)

  # correct number of extracted elements
  #assert file_count == 79458, f"Expected 79458 file extracted, got {file_count} "

  # we are starting splitting the actual dataset in smaller chunks

  IMAGE_SPLIT_PATH = os.path.join(DATASET_PATH, dataset, 'images')
  LABEL_SPLIT_PATH = os.path.join(DATASET_PATH, dataset, 'labels')
  #EXAMPLE /content/SAR-Ship-Dataset/ship_dataset_v0
  EXT_DATASET_PATH = os.path.join(DATASET_PATH, dataset)

  # create ./images and ./labels
  try:
    os.makedirs(IMAGE_SPLIT_PATH)
    os.makedirs(LABEL_SPLIT_PATH)
  except Exception as e:
    print(f"Some error occured: {e}")


  # get all files that ends with .jpg in ./SAR-Ship-Dataset/ship_dataset_v0
  # and transfer them to ./SAR-Ship-Dataset/ship_dataset_v0/images
  for file in os.listdir(EXT_DATASET_PATH):
      if file.endswith('.jpg'):
          shutil.move(os.path.join(EXT_DATASET_PATH, file),
                      os.path.join(IMAGE_SPLIT_PATH, file))
      elif file.endswith('.txt'):
          shutil.move(os.path.join(EXT_DATASET_PATH, file),
                      os.path.join(LABEL_SPLIT_PATH, file))


  # take 70% of images in ./images and put them in ./train/images and the corresponding .txt from ./labels in ./train/labels
  # take 30% of images in ./images and put them in ./VALIDATION

  def create_train_test_valid(num_img_to_pick=-1):
    # get all images in ./images
    # IMG_SPLIT_PATH is this: ./SAR-Ship-Dataset/ship_dataset_v0/images
    images = os.listdir(IMAGE_SPLIT_PATH)
    size = 0 # number of items to take from images
    if(num_img_to_pick == -1):
      size = len(images)
    else:
      size = num_img_to_pick
    # shuffle images
    random.shuffle(images)

    # get 70% of images for training
    train = images[:int(size*0.7)]
    # get 30% of images for validation
    validation = images[int(size*0.7):size]

    # create train, test, valid and add the images and labels

    def copy_in_new_folder(split, list_img):
      split_img_path = os.path.join(DATASET_PATH, dataset, split, "images")
      split_labels_path = os.path.join(DATASET_PATH, dataset, split, "labels")
      os.makedirs(os.path.join(DATASET_PATH, dataset, split))
      os.makedirs(split_img_path)
      os.makedirs(split_labels_path)
      for img in list_img:
        shutil.move(IMAGE_SPLIT_PATH+'/'+img, split_img_path+'/'+img)
        shutil.move(LABEL_SPLIT_PATH+'/'+img[:-4]+'.txt',
                    split_labels_path+'/'+img[:-4]+'.txt')

    copy_in_new_folder("train", train)
    copy_in_new_folder("validation", validation)

    shutil.rmtree(IMAGE_SPLIT_PATH)
    shutil.rmtree(LABEL_SPLIT_PATH)
    print(f'✅ Dataset {dataset} ready for training and validation!')

  create_train_test_valid(num_img_to_pick = nmr_images)

✅ Dataset ship_dataset_preprocessed ready for training and validation!
✅ Dataset ship_dataset_v0 ready for training and validation!
✅ Dataset ship_dataset_leefilter ready for training and validation!


### Dataset training

In [ ]:
# TODO epochs here and in the following can be parametrized using ClearML
# let's see how to do it

task.connect(args)
# data.yaml is downloaded from the github repo
data_train = model.train(**args) #train the model
task.close()

Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/SAR-Ship-Dataset/data_leefilter.yaml, epochs=100, time=None, patience=50, batch=256, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

100%|██████████| 755k/755k [00:00<00:00, 16.0MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3011043 parameters, 3011027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/SAR-Ship-Dataset/ship_dataset_leefilter/train/labels... 7000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7000/7000 [00:03<00:00, 1973.20it/s]


...

    100/100      5.86G      1.046     0.4877     0.9322         96        256: 100%|██████████| 28/28 [00:28<00:00,  1.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:14<00:00,  2.43s/it]

                   all       3000       3974       0.95      0.952      0.981      0.691



100 epochs completed in 1.280 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:18<00:00,  3.02s/it]


                   all       3000       3974      0.951      0.952      0.981      0.691
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train


### Save locally the result of the different runs
No need to run this if we connect with ClearML

In [ ]:
def save_all_runs_locally():
  def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
  locale.getpreferredencoding = getpreferredencoding
  !zip -r /content/runs.zip /content/runs/

  files.download('/content/runs.zip')
#save_all_runs_locally()

### Validation process

In [ ]:
# PERFORM VALIDATION OF THE MODEL

data_validation = model.val()

### Predict new data

In [ ]:
# This is used for having a prediction on new data
# It is not mandatory but useful to test inference on new data
# In this we can extract from the model the actual folder where the weights
# are saved for retriving the best or use clearml to run inference
'''
model = YOLO(PROJECT_LOC+"best.pt") # random best da cambiare con il training approfondito
results = model.predict(
    source    = PROJECT_LOC+"test_images",
    conf      = 0.5,  # confidence > 50%
    #save      = True, # this will save the images with annotation in /runs/detect/predict/file_name.xyz
    save_txt  = True, # this will save the actual labels in /runs/detect/predict/labels/<original_file_name>.txt
    imgsz     = 256)
'''

### Future improvements

In [ ]:
# FUTURE: benchmark to watch what is happening in the edge?
# FUTURE: tuning?